# Zoptymalizowana symulacja plazmowa Particle in Cell w Pythonie

* Dominik Stańczak
* opiekun badań: dr Sławomir Jabłoński, IFPiLM
* kierujący pracą: dr inż Daniel Kikoła

# Plazma - *po co* ją symulować?
* cała dziedzina fizyki skupiona na zastosowaniach - głównie do fuzji termojądrowej
    * ![https://en.wikipedia.org/wiki/File:Fusion_rxnrate.svg](prezentacja_sem_dyp/img/fusion_temperature_plot.png)
    * Potrzebujemy ogromnych temperatur, by ją osiągnąć - osiągalne jedynie w plazmach
    * Również inne zastosowania - akceleratory (CERNowski projekt *AWAKE*), silniki kosmiczne (Halla), obróbka procesorów, medycyna...

* Wyjątkowo złożony układ: nie znamy analitycznych rozwiązań na praktycznie żaden z ciekawszych przypadków
* Główny koncepcyjny problem: liczba dalekozasięgowych oddziaływań w układzie rośnie jak $n^2$ w liczbie cząstek

# Plazma - *jak* ją symulować?

### Równanie Vlasova - podstawowy, najściślejszy aparat (tu: w uproszczonej elektrostatycznej wersji)

$$ \frac{\partial f_\alpha}{\partial t} + \vec{v}_\alpha \cdot \frac{\partial f_\alpha}{\partial \vec{x}} + \frac{q_\alpha \vec{E}}{m_\alpha} \cdot \frac{\partial f_\alpha}{\partial \vec{v}} = 0$$

$f$ jest gęstością cząstek w danym punkcie w przestrzeni fazowej. $\alpha$ oznacza konkretny typ cząstek. $E$ to pole elektryczne, $q$ - ładunek, $m$ - masa.

### Problemy z równianiem Vlasova

W jednej chwili czasu pracujemy na 6-wymiarowej siatce. Więc problem skaluje się jak $N_x^3N_v^3$. Siatka musi mieć dobrą rozdzielczość i zasięg (zjawisko wysokoonergetycznego ogona elektronowego).

> Superkomputer Titan w Oak Ridge National Laboratory, USA ma 27 petaflopów wydajności obliczeniowej i zużywa do 9 megawatów mocy. Bez maszyny na skalę Titana sensownie nie da się czegoś takiego zrobić.

## Przybliżenia
Oczywiście pomijają część efektów, np. magnetohydrodynamika
* nie działa daleko od równowagowej dystrybucji
* wymaga, aby plazma była zdominowana przez kolizje
* zupełnie się nie nadaje do symulacji plazmy termojądrowej

Oraz są modele opierające się na fundamentalnej fizyce ruchu cząstek według równań Newtona, jak na przykład...

# Modele Particle in Cell

In [1]:
python plotting.py 1default.hdf5 -dev

SyntaxError: invalid syntax (<ipython-input-1-517f5ac688e4>, line 1)

## Główny algorytm
Symulacje cząsteczkowe są stosunkowo łatwe ze względu na ich **liniowe skalowanie w liczbie cząstek**. Problematycznym krokiem jest jedynie obliczanie oddziaływań - w przypadku ogólnym, gdy każda cząstka może oddziaływać na każdą, liczba oddziaływań skaluje się jak $n^2$.

Jeśli więc (upraszczając) nasz zasób pamięć RAM wystarczy na milion cząstek, to na dwa miliony będziemy potrzebować cztery razy tyle RAMu. Milion zaś to niewiele!

### Czy musimy obliczać siły bezpośrednio?

Zdefiniujmy sobie dyskretną siatkę o liczbie komórek $n_g << n$. Każda z cząstek wpada w którąś komórkę.

Policzmy, ile jest cząstek każdego rodzaju w każdej komórce. Sumując ładunki w komórce, dostajemy gęstość ładunku. Złożoność $O(n)$ - wykonujemy tą operację raz dla każdej cząstki. To tzw. **depozycja ładunku**. *W rzeczywistości - interpolacja do krawędzi siatki!*

Mamy więc na $n_g$ komórek siatki zdefiniowaną gęstość ładunku. Możemy teraz wykorzystać równanie Poissona:

$$\frac{\rho}{\varepsilon_0} = \nabla E = -\nabla^2 \varphi$$

Ze znanym $\rho$ można wykorzystać metody numeryczne jak

* Jacobiego
* Gaussa-Seidela
* Successive Over-relaxation
* Conjugate Gradients
* **metody spektralne**, wykorzystujące transformaty Fouriera *(algorytm FFT w przypadkach dyskretnych!)*

aby obliczyć $E$ - ze złożonością zazwyczaj faktycznie kwadratową *(dowód trywialny)*, ale w $n_g$ - zaś jeśli $n_g << n$, to $n_g^2 << n^2$! **Koszt obliczenia rzędu $n_g^2$ nie jest taki straszny!**

### Mamy więc obliczone pole elektryczne - ale na siatce. Co z tego?

Tyle, iż teraz możemy zinterpolować wartości pola z siatki do cząstek, raz na cząstkę - $O(n)$, wciąż mniej niż kwadratowo!

### Mamy więc siły działające na cząstki

Teraz zaś już trywialną kwestią jest użyć ich oraz znanych uprzednio pędów, by zaktualizować pędy i położenia cząstek do kolejnej iteracji. (tzw. **particle push**)

Przesunięte na nowo cząstki można zaś wykorzystać do znalezienia nowego rozkładu ładunku... I wracamy do punktu wyjścia!

# Prawdopodobnie najważniejszy fakt czyniący symulacje PIC sensownymi:

W równaniach ruchu oddziaływania elektryczne skalują się jak $\frac{q}{m} = \frac{Nq}{Nm}$. Jedna cząstka obliczeniowa może reprezentować wiele cząstek rzeczywistych! To pozwala nam symulować plazmę makroskalową!

# Stan obecny prac

* Korzystam jak dotąd głównie z książki Birdsalla & Langdona z 1975, *Plasma Physics via Computer Simulation* oraz materiałów (m. in. wykładów) dostępnych w Internecie
* Docelowo ma być elektromagnetyczna i symulować oddziaływanie plazmy z laserem, więc przypadek relatywistyczny 
* Model na tym etapie jest elektrostatyczny, okresowy (nieskończona plazma), nierelatywistyczny (pracuję na prędkościach i przyjmuję $\gamma = 1$)

# Specyfika symulacji
* Jeden wymiar, ale jest napisana tak, by łatwo było ją rozbudować na więcej
* *field solver* jest obecnie spektralny, wykorzystuje dane z całej siatki (globalnie). *To się kłóci z relatywistyką!*
* *Particle pusher* to tzw. algorytm Borysa, podobny do algorytmu Verleta - w przypadku jednowymiarowym elektrostatycznym redukuje się do klasycznego *leapfroga*.
* Obecnie: okresowe warunki brzegowe, cząstki wychodzące z prawej wchodzą od lewej.

# Problemy, które do tej pory napotkałem
* Zaczynanie kodu od początku zamiast rozsądnej przeróbki tego, co mam
* Mnóstwo błędów wynikających z **nieprzemyślanego dostępu do zmiennych**
* Obliczanie pola oraz energii - **skalowanie**, problem utworzenia wektora $k$

# Co pozostało do zrobienia:

* dobudować pole magnetyczne w pusherze oraz field solverze
* przejść na solver i pusher relatywistyczny
    * lokalny! Wrócimy do tego wątku później...
* Dodać laser (jako warunek brzegowy zależny od czasu na pola $E$, $B$)
* Testy prędkości i optymalizacja - próby wyciągnięcia maksymalnej możliwej prędkości

# Optymalizacja kodu
### Numpy
* Python jest uznawany za "wolny" - ale C i Fortran nie są.
* Python jest wybitnym językiem "klejącym". Odwołując się z poziomu Pythona do gotowych implementacji operacji wektorowych w innych architekturach, możemy osiągnąć szybkości praktycznie takie same jak w językach niskopoziomowych
* Standardową biblioteką całego *scientific computing* w Pythonie jest Numpy, implementujący operacje wektorowe

In [25]:
import numpy as np

N = 1000
x = np.random.random(N)

def odleglosci_sekwencyjnie(x):
    r = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            d = x[i] - x[j]
            r[i, j] = d
            r[j, i] = -d
    return r

odleglosci_sekwencyjnie(x)[:4,:4]

array([[-0.        , -0.08215605,  0.30792906,  0.4512894 ],
       [ 0.08215605, -0.        ,  0.39008511,  0.53344545],
       [-0.30792906, -0.39008511, -0.        ,  0.14336034],
       [-0.4512894 , -0.53344545, -0.14336034, -0.        ]])

In [26]:
def odleglosci_numpy(x):
    return x.reshape(N, 1) - x.reshape(1, N)

odleglosci_numpy(x)[:4,:4]

array([[ 0.        , -0.08215605,  0.30792906,  0.4512894 ],
       [ 0.08215605,  0.        ,  0.39008511,  0.53344545],
       [-0.30792906, -0.39008511,  0.        ,  0.14336034],
       [-0.4512894 , -0.53344545, -0.14336034,  0.        ]])

In [29]:
%timeit odleglosci_sekwencyjnie(x)

1 loop, best of 3: 335 ms per loop


In [30]:
%timeit odleglosci_numpy(x)

100 loops, best of 3: 3.59 ms per loop


### Numba

Kompilacja *just-in-time* sekwencyjnego kodu Pythona do języków niskopoziomowych i odwoływanie się do takiej, zapisanej implementacji

In [34]:
from numba import njit

@njit() # <- tak, to jest jedyna różnica
def odleglosci_sekwencyjnie_numba(x):
    r = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            d = x[i] - x[j]
            r[i, j] = d
            r[j, i] = -d
    return r

odleglosci_sekwencyjnie_numba(x)[:4,:4]

array([[-0.        , -0.08215605,  0.30792906,  0.4512894 ],
       [ 0.08215605, -0.        ,  0.39008511,  0.53344545],
       [-0.30792906, -0.39008511, -0.        ,  0.14336034],
       [-0.4512894 , -0.53344545, -0.14336034, -0.        ]])

In [35]:
%timeit odleglosci_sekwencyjnie_numba(x)

100 loops, best of 3: 7.01 ms per loop


### Dygresja teoretyczna - paralelizm w obliczeniach
* Problemy trywialnie paralelizowalne
* Złożone podejścia do paralelizmu
* Karty graficzne!
* Kopiowanie pamięci

## PyCUDA

PIC jest dlatego przyszłościowy, że jest *trywialnie paralelizowalny* - każda z cząstek jest niezależna od siebie, jedynie od siatki i pola.

Istnieją implementacje obliczeń macierzowych z Numpy wykorzystujące karty graficzne Nvidii.

Numpy osiąga paralelizm na procesorze (przykładowo, 4 rdzenie dają 4 wątki) - przykładowa starzejąca się karta Nvidii pozwala uruchomić 1024 wątki.

PIC jest więc *cudownym* przykładem programu do uruchomienia kart graficznych!

Relatywistyczny *field solver* jest zazwyczaj lokalny - to też pozwala uruchomić go na karcie graficznej!